In [1]:
import re
import json
from collections import defaultdict
from uuid import uuid4

def parse_m3u_to_json(m3u_file_path, provider_name="My IPTV", provider_url="https://example.com/m3u.json", provider_logo=None):
    with open(m3u_file_path, "r", encoding="utf-8") as f:
        lines = [line.strip() for line in f if line.strip()]

    provider_id = "provider-" + str(uuid4())
    json_data = {
        "id": provider_id,
        "name": provider_name,
        "description": f"A list of channels from {provider_name}",
        "url": provider_url,
        "color": "#181818",
        "image": {
            "url": provider_logo or "",
            "type": "contain",
            "height": 187,
            "width": 28
        },
        "grid_number": 2,
        "groups": []
    }

    groups = defaultdict(list)

    # Regex to extract info from #EXTINF line
    extinf_regex = re.compile(r'#EXTINF:-1(?:\s+tvg-logo="(?P<logo>.*?)")?(?:\s+group-title="(?P<group>.*?)")?,(?P<name>.*)')

    for i in range(len(lines)):
        line = lines[i]
        if line.startswith("#EXTINF"):
            match = extinf_regex.match(line)
            if match:
                logo = match.group("logo")
                group = match.group("group") or "Others"
                name = match.group("name")
                # next line should be the URL
                url = lines[i + 1] if i + 1 < len(lines) else ""
                groups[group].append({
                    "id": str(uuid4()),
                    "name": name,
                    "description": "",
                    "label": "Full",
                    "image": {
                        "url": logo or "",
                        "type": "contain",
                        "width": 1920,
                        "height": 1080
                    },
                    "display": "default",
                    "type": "single",
                    "enable_detail": True,
                    "sources": [
                        {
                            "id": str(uuid4()),
                            "name": "Source 1",
                            "image": None,
                            "contents": [
                                {
                                    "id": str(uuid4()),
                                    "name": "Content 1",
                                    "image": None,
                                    "streams": [
                                        {
                                            "id": str(uuid4()),
                                            "name": "Full",
                                            "image": {
                                                "url": logo or "",
                                                "type": "contain",
                                                "width": 1920,
                                                "height": 1080
                                            },
                                            "stream_links": [
                                                {
                                                    "id": str(uuid4()),
                                                    "name": "Server 1",
                                                    "url": url,
                                                    "type": "hls",
                                                    "default": True,
                                                    "enableP2P": True,
                                                    "subtitles": None,
                                                    "remote_data": None,
                                                    "request_headers": None,
                                                    "comments": None
                                                }
                                            ]
                                        }
                                    ]
                                }
                            ],
                            "remote_data": None
                        }
                    ]
                })

    # Convert groups dictionary to the list format
    for group_name, channels in groups.items():
        json_data["groups"].append({
            "id": "group-" + str(uuid4()),
            "name": group_name,
            "display": "vertical",
            "image": {
                "url": "",
                "type": "contain",
                "width": 1920,
                "height": 1080
            },
            "grid_number": 1,
            "enable_detail": False,
            "channels": channels
        })

    return json_data

# Example usage
m3u_file = "tung_iptv.m3u"
json_result = parse_m3u_to_json(m3u_file, provider_name="Tung IPTV", provider_logo="https://raw.githubusercontent.com/lamtung16/iptv/refs/heads/main/images/logo.png")

# Save to file
with open("tung_iptv.json", "w", encoding="utf-8") as f:
    json.dump(json_result, f, ensure_ascii=False, indent=2)

print("Conversion complete! JSON saved as iptv.json")

Conversion complete! JSON saved as iptv.json
